In [7]:
import requests
from requests import Response
aqi_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
response:Response=requests.get(aqi_url)
if response.status_code==200:
    print('下載成功')
else:
    print('下載失敗')

下載成功


In [8]:
#得到python的資料結構(list或dict)
data_base:dict=response.json()

In [16]:
#json的字串格式
data_json=response.text

In [20]:
from pydantic import BaseModel, Field, field_validator
class Site(BaseModel):
    站點:str=Field(alias='sitename')
    縣市:str=Field(alias='county')
    aqi:int=Field(alias='aqi')
    pm25:float=Field(alias='pm2.5_avg')
    時間:str=Field(alias='datacreationdate')
    #解決model_validate_json空字串的問題
    @field_validator('pm25', mode='before')
    @classmethod
    def sapce_zero(cls, value):
        if value=='':
            value=0.0
        return value

class AQI(BaseModel):
    records:list[Site] #records名字要和資料的一樣(json viewer)


In [ ]:
aqi=AQI.model_validate_json(data_json)
#source_data=aqi.model_dump()
source_data=aqi.model_dump()['records'] #(aqi.model_dump()的key是records)
source_data

In [31]:
import pandas as pd
df1=pd.DataFrame(source_data)
df1

,站點,縣市,aqi,pm25,時間
0,屏東(枋山),屏東縣,23,0.0,2024-07-08 22:00
1,大甲（日南國小）,臺中市,22,5.0,2024-07-08 22:00
2,新北(樹林),新北市,22,6.3,2024-07-08 22:00
3,屏東（琉球）,屏東縣,30,9.2,2024-07-08 22:00
4,臺南（麻豆）,臺南市,23,5.0,2024-07-08 22:00
...,...,...,...,...,...
995,頭份,苗栗縣,24,7.3,2024-07-08 11:00
996,新竹,新竹市,16,4.0,2024-07-08 11:00
997,竹東,新竹縣,18,5.5,2024-07-08 11:00
998,湖口,新竹縣,24,7.5,2024-07-08 11:00
